# Create an HHS Region to State Crosswalk

In [33]:
import pandas as pd

# Text from https://www.hhs.gov/about/agencies/iea/regional-offices/index.html
with open("hhs.txt") as f:
    hhs_state_pairs = []
    for line in f.readlines():
        if "Region" in line:
            region_number = int(line[7:9])
        else:
            states = line.replace(" and", "").strip().split(", ")
            for state in states:
                hhs_state_pairs.append((region_number, state))

# Make naming adjustments
hhs_state_pairs.remove((2, "the Virgin Islands"))
hhs_state_pairs.append((2, "U.S. Virgin Islands"))
hhs_state_pairs.remove((9, "Commonwealth of the Northern Mariana Islands"))
hhs_state_pairs.append((9, "Northern Mariana Islands"))

# Form dataframe
hhs_df = pd.DataFrame(hhs_state_pairs, columns=["hhs_region_number", "state_name"])
hhs_df['hhs_region_number'] = hhs_df['hhs_region_number'].astype(str)
hhs_df.head()

,hhs_region_number,state_name
0,1,Connecticut
1,1,Maine
2,1,Massachusetts
3,1,New Hampshire
4,1,Rhode Island


In [34]:
# Open state codes crosswalk from the US Census Bureau.
states_df = (
    pd.read_csv("http://www2.census.gov/geo/docs/reference/state.txt?#", delimiter="|")
    .drop(columns="STATENS")
    .rename(columns={"STATE": "state_code", "STUSAB": "state_id", "STATE_NAME": "state_name"})
)
states_df["state_code"] = states_df["state_code"].astype(str).str.zfill(2)
states_df["state_id"] = states_df["state_id"].astype(str).str.lower()

state_territories = pd.DataFrame([
    {
        "state_code": 70,
        "state_name": "Republic of Palau",
        "state_id": "pw",
    },
    {
        "state_code": 68,
        "state_name": "Marshall Islands",
        "state_id": "mh",
    },
    {
        "state_code": 64,
        "state_name": "Federated States of Micronesia",
        "state_id": "fm",
    },
])
# Add a few extra US state territories manually
states_df = pd.concat([states_df, state_territories])
states_df.head()


,state_code,state_id,state_name
0,01,al,Alabama
1,02,ak,Alaska
2,04,az,Arizona
3,05,ar,Arkansas
4,06,ca,California


In [35]:
merged_df = states_df.merge(hhs_df, on="state_name", how="left").dropna()
merged_df.head()

,state_code,state_id,state_name,hhs_region_number
0,01,al,Alabama,4
1,02,ak,Alaska,10
2,04,az,Arizona,9
3,05,ar,Arkansas,6
4,06,ca,California,9


In [36]:
merged_df.to_csv("state_hhs_crosswalk.csv", index=False)